# Введение. Исследование надёжности заёмщиков
<div class="alert alert-block alert-info">
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение, количество детей, уровень зароботка и цель оформления кредита клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов. В датасете содержится информация о стаже, возрасте, детях, образовании, семейном положении, поле, должности, наличию долгов по возврату кредитов в срок, доход и цель оформления кредита по каждому из клиентов. Использовать мы будем только информацию о детях, семейном положении, уровне дохода, цели оформления кредита и наличию долгов.

Будет разработан индекс, который позволит рассчитать уровень возвращаемости кредитов в срок. 

План работы: 
1. [Импорт библиотек и знакомство с файлом](#start)
2. [Предобработка данных:](#processing)
    * [Обработка пропусков](#processing1)
    * [Замена типа данных](#processing2)
    * [Обработка дубликатов](#processing3)
    * [Лемматизация](#processing4)
    * [Категоризация данных](#processing5)
    <br>
    <br>
3. [Рассчёт индекса. Поиск зависимостей](#сoun_of_index)
4. [Общий вывод](#conclusion)<br>

<br>
На основании исследования будут выделены группы, наиболее предпочтительные для выдачи кредита, и группы, кредит которым следует давать с осторожностью. <br>
<br>
Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Импорт библиотек и знакомство с файлом
<a id="start"></a>

In [1]:
import pandas as pd
from collections import Counter
from pymystem3 import Mystem
m = Mystem()


<div class="alert alert-block alert-info">
Собрал импорты и константы в одном месте. Открытие датасета убрал в следующий блок.

In [2]:
credit_data = pd.read_csv('/datasets/data.csv')
credit_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
credit_data.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [4]:
credit_data.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10263,0,-1779.684580,36,среднее,1,женат / замужем,0,M,сотрудник,0,203973.635751,заняться образованием
8521,0,-381.897136,25,неоконченное высшее,2,в разводе,3,F,сотрудник,0,79454.160535,покупка недвижимости
13660,1,-88.280875,44,высшее,0,женат / замужем,0,M,сотрудник,0,155906.409139,получение образования
4722,0,-2147.094331,44,среднее,1,в разводе,3,M,компаньон,0,388633.861914,покупка коммерческой недвижимости
6826,1,-2294.481990,40,среднее,1,женат / замужем,0,M,сотрудник,0,253374.870249,сделка с автомобилем
9823,0,-831.616198,30,среднее,1,женат / замужем,0,M,компаньон,0,141965.355510,заняться образованием
20962,1,-2589.669758,31,высшее,0,женат / замужем,0,M,компаньон,0,166323.689177,автомобиль
4332,0,-347.677186,36,высшее,0,Не женат / не замужем,4,F,сотрудник,0,73137.234205,строительство собственной недвижимости
3646,0,362739.565850,69,высшее,0,женат / замужем,0,F,пенсионер,0,314654.970464,покупка недвижимости
1539,0,-15785.678893,59,высшее,0,Не женат / не замужем,4,F,сотрудник,0,119563.851852,операции с коммерческой недвижимостью


<div class="alert alert-block alert-info">
    Попробовал)</div>

In [5]:
credit_data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


<div class="alert alert-block alert-info">
Использовал метод describe(), для поиска анамалий. <br>
<br>
* Видно пустые значения в столбиках total_income и days_employed<br>
* Отрицательные и экстремально высокие значения в столбике days_employed<br>
* Нулевые значения в графе dob_years<br>
   


Отрицательные значения в столбике 'days_employed'. Стаж пользователя credit_data.loc[4] составляет 932 года. Разный регистр в столбике 'education'.

In [6]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пустые значения в столбиках 'days_employed' и 'total_income'

In [7]:
credit_data[credit_data['total_income'].isna()].income_type.value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [8]:
credit_data.income_type.value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Пропуски составляют примерно 10% по каждому из источников дохода. 

In [9]:
credit_data[credit_data['total_income'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Пустые значения в столбиках 'days_employed' и 'total_income' совпадают. 

**Вывод**

При первом взгляде на датафрейм выявляется ряд проблем, которые могут помешать анализу данных: 
- Отрицательные значения в столбике стажа (days_employed)
- Смущение вызывает и дробный формат данных, где подсчёт идёт в целых днях
- Стаж пользователя credit_data.loc[4] составляет 932 года
- Пустые ячейки в столбиках 'days_employed' и 'total_income'
- Разный регистр у одних значений в столбике образования
- В графе children есть отрицательные значения и семьи с 20 детьми


Строчки с пустыми значениями в столбике 'total_income' и 'days_employed' совпадают. Либо эти клиенты не оставили данные о своём месте работы и уровне зароботка, либо данные потерялись. Клиенты с нулевым уровнем дохода быть не могут, т.к. среди них есть пенсионеры, которые получают гарантированную пенсию от государства.

Характер данных в столбике 'days_employed' говорит о том,  что данные импортировались некорректно, однако этот столбик для поставленных задач не нужен.              

Процент пропущенных значений на уровне 10%. Не хотелось бы потерять эти данные. Будем менять пустые значения в столбике 'total_income'на медианные по типу дохода. 

В столбике 'children' содержатся отрицательные значения и строчки, где количество детей 20. Семей с таким количеством детей не может быть много, но в таблице содержится много таких строк, что говорит об опечатке. Скорее всего речь идёт о 2 детях в семье. 
Отрицательные значения переведем в положительне. 


## Предобработка данных 
<a id="processing"></a>

### Обработка пропусков
<a id="processing1"></a>

In [10]:
empty_data_list = credit_data[credit_data['total_income'].isna()].income_type.unique()

for element in empty_data_list:
    median_income = credit_data[credit_data['income_type']==element].total_income.median()
    credit_data.loc[credit_data['income_type']==element, 'total_income'] = \
    credit_data.loc[credit_data['income_type']==element, 'total_income'].fillna(median_income)

Создали цикл для замены пустых значений медианными по каждому источнику дохода.

In [11]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пустых значений в столбике 'total_income' не осталось.

**Вывод**

Мы создали цикл, который определяет пустые значения в колонке 'total_income' и подставляет медианные значения по типу источника дохода. 

### Замена типа данных
<a id="processing2"></a>

In [12]:
credit_data.total_income = credit_data.total_income.astype('int')

In [13]:
credit_data['children'] = credit_data['children'].apply(abs)

Заменили отрицательное значение в графе 'children'

In [14]:
credit_data.loc[credit_data['children']==20,'children']=2

Присвоили строчкам, в которых было указано 20 детей, значение 2. 

In [15]:
credit_data.education =credit_data.education.apply(str.lower)
credit_data.family_status =credit_data.family_status.apply(str.lower)
credit_data.gender =credit_data.gender.apply(str.lower)
credit_data.income_type =credit_data.income_type.apply(str.lower)
credit_data.purpose =credit_data.purpose.apply(str.lower)

<div class="alert alert-block alert-info">
Перевёл регистр на нижний во всех столбиках типа object. Теперь нашёл 71 дубликат. 

**Вывод**

Заменили вещественный тип (float64) на целочисленный (int) методом astype() в столбике 'total_income'. 
Пропустили столбик days_employed, т.к. он не используется для анализа. Обработали ошибки с столбике 'children'

### Обработка дубликатов
<a id="processing3"></a>

In [16]:
credit_data.duplicated().sum()

71

Обнаружили 54 дублирующиеся строки. Удалим их.

In [17]:
credit_data = credit_data.drop_duplicates().reset_index(drop=True)
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       19351 non-null float64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


<div class="alert alert-block alert-info">
Удалил дубликаты с учетом регистра. 

**Вывод**

Все дублирующиеся значения удалены. Метод drop_duplicates() удаляет из датафрейма повторящиеся строки, при этом у строк остаются прежние индексы. Чтобы проиндексировать строки заново, я использовал метод reset_index() с параметром drop=True, чтобы старые индексы удалились. 

Скорее всего, причина появляения дубликатов здесь - ошибка на стороне сервера, который задвоил часть хранимых данных.


### Лемматизация
<a id="processing4"></a>

In [18]:
credit_data.purpose.value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [19]:
lemmas = []
for row in range (len(credit_data)):
    lemmas += m.lemmatize(credit_data.loc[row,'purpose'])
Counter(lemmas)

Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         '\n': 21454,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

Посчитали частоту встречаемости для каждого слова. Посчитали количество уникальных фраз в столбике 'purpose'.

Можно выделить основные цели, для которых берут кредит: 
   * Проведение свадьбы
   * Операциии с недвижимостью
   * Получение образования
   * Сделка с автомобилем
   
Далее категоризируем цели приобретения кредита по этим пунктам. 

In [20]:
def category_purpose(purpose):
    lemmed_word = m.lemmatize(purpose)
   
    for word in lemmed_word: 
        if word == 'жилье':
            return 'операциии с недвижимостью'
        if word == 'недвижимость':
            return 'операциии с недвижимостью'
        if word == 'свадьба':
            return 'проведение свадьбы'
        if word == 'автомобиль':
            return 'сделка с автомобилем'
        if word == 'образование':
            return 'получение образования'  

In [21]:
credit_data['category'] = credit_data['purpose'].apply(category_purpose)

Применили полученную функцию к столбику 'purpose'. Результат сохранили в новом столбике 'category'.

In [22]:
credit_data.category.value_counts()

операциии с недвижимостью    10811
сделка с автомобилем          4306
получение образования         4013
проведение свадьбы            2324
Name: category, dtype: int64

Все строчки в новом столбике заполнены. 

**Вывод**

В ходе лемматизации мы разбили столбик 'purpose' по ключевым словам. Мы изучили самые часто встречающиеся слова и просмотрели все фразы. На основании этого мы выделили 4 основные категори целей приобретения кредита и закодировали столбик 'purpose'.

### Категоризация данных
<a id="processing5"></a>

In [23]:
def cotegorize_children(children):
    if children == 0:
        return 'бездетные'
    if 0 < children < 3:
        return 'малодетные'
    if children > 2:
        return 'многодетные'

In [24]:
credit_data['children_category'] = credit_data['children'].apply(cotegorize_children)

Категоризировали клиентов по количеству детей в семье. Использовали следующие деление: 1) бездетные; 2) малодетные; 3) многодетные;

In [25]:
round(credit_data.total_income.describe())

count      21454.0
mean      165320.0
std        98187.0
min        20667.0
25%       107623.0
50%       142594.0
75%       195820.0
max      2265604.0
Name: total_income, dtype: float64

Разобьём пользователей по уровню дохода. Разделим данныепо медиане и 1 и 3 квартели на 4 группы:
* Меньше 107654 - "бедные"
* От 107654 до 142594 - "ниже_среднего"
* От 142594 до 195768 - "выше_среднего"
* От 195768 - "богатые"

In [26]:
def cotegorize_income(income):
    if income < 107654:
        return 'бедные'
    if 107654 <= income < 142594:
        return 'ниже_среднего'
    if 142594 <= income <195768:
        return 'выше_среднего'
    if income >= 195768:
        return 'богатые'

In [27]:
credit_data['income_category'] = credit_data['total_income'].apply(cotegorize_income)

Категоризировали пользователей по уровню дохода.

In [28]:
children_debt_index = credit_data.groupby('children_category').debt.mean()
children_debt_index

children_category
бездетные      0.075438
малодетные     0.092654
многодетные    0.081579
Name: debt, dtype: float64

Посчитали индекс возврата кредитов по категориям наличия детей. 

In [29]:
family_debt_index = credit_data.groupby('family_status').debt.mean()
family_debt_index

family_status
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093471
женат / замужем          0.075452
не женат / не замужем    0.097509
Name: debt, dtype: float64

In [30]:
income_debt_index = credit_data.groupby('income_category').debt.mean()
income_debt_index

income_category
бедные           0.079545
богатые          0.071349
выше_среднего    0.086488
ниже_среднего    0.087401
Name: debt, dtype: float64

In [31]:
purpose_debt_index = credit_data.groupby('category').debt.mean()
purpose_debt_index

category
операциии с недвижимостью    0.072334
получение образования        0.092200
проведение свадьбы           0.080034
сделка с автомобилем         0.093590
Name: debt, dtype: float64

**Вывод**

В ходе этого этапе мы создали категории по уровню дохода и наличию детей. Далее мы рассчитали индекс возврата кредитов для разных категорий клиентов: по семейному статусу, по наличию детей, по уровню дохода, по цели кредита. 

## Подсчёт индекса. Поиск зависимостей
<a id="сoun_of_index"></a>

**1) Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [32]:
children_debt_index

children_category
бездетные      0.075438
малодетные     0.092654
многодетные    0.081579
Name: debt, dtype: float64

**Вывод**

Зависимость между наличием детей и возвратом кредита прослеживается. Клиенты без детей реже имеют задолженности по возврату кредитов. 

Бездетные чаще отдают кредиты, т.к. они менее финансово нагружены. 

При этом среди родителей чаще возвращают кредиты в срок те, кто имеет 3 и более ребенка. Это связано с тем, что 3 ребенок и выше чаще появляются после 30 лет, когда финансовое положение семьи стабильное.

**2) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [33]:
income_debt_index

income_category
бедные           0.079545
богатые          0.071349
выше_среднего    0.086488
ниже_среднего    0.087401
Name: debt, dtype: float64

**Вывод**

Зависимость в уровне дохода тоже прослеживается - чаще кредиты всрок возвращают бедные и богатые клиенты. 

Как ни странно, пользователи со средним уровнем дохода чаще задерживают выплоты по кредиту. Это может быть связано с тем, что в данной категории людей люди меньше бояться брать кредиты, но иногда могут плохо рассчитывать свои силы и не выплачивать в срок.  

Бедные же берут кредиты в крайних случаях под жизненно необходимые вещи. Они осознают угрозу, что могут невплатить кредит в срок и поэтому тщательно обдумывают такое решение. 

В целом же индекс по категориям уровня дохода отличается не сильно. 

**3) Как разные цели кредита влияют на его возврат в срок?**

In [34]:
purpose_debt_index

category
операциии с недвижимостью    0.072334
получение образования        0.092200
проведение свадьбы           0.080034
сделка с автомобилем         0.093590
Name: debt, dtype: float64

**Вывод**

Хуже отдают кредит, взятые с целью получить образование и на покупку автомобиля. 

При получении образования это говорит о том, что люди либо ещё в поисках работы либо хотят сменить профессию. Это говорит об их неустойчивом финансовом положении. 

Автомобиль же в России является показателем статуса, и многие люди приобретают его в кредит, желая поднять свой социальный статус, однако не имея для этого финансовых возможностей. 

На свадьбы берут не такие большие суммы, чтобы их было сложно возвращать. И в России принято дарить деньги на свадьбу, что создаёт финансовую подушку после её проведения, поэтому меньше сложностей с выплатой кредита после. 

При операциях с недвижимостью гарантией выплаты кредита является приобретаемая собственность, поэтому люди, берущие кредит с этой целью, больше мотивированы его отдать. 

**4) Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [35]:
family_debt_index

family_status
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093471
женат / замужем          0.075452
не женат / не замужем    0.097509
Name: debt, dtype: float64

**Вывод**

Клиенты, кто сейчас находится или был в официальном браке чаще отдают кредиты, чем те, кто не женат/не замужем или разведен. 

Супруги несут юридические обязательства друг за друга при выплате кредита и поэтому больше помогают своим половинкам. 
Также можно добавить, что у супругов выше уровень ответственности друг за друга, т.к. они являются семьей. 

Вдовцы и люди в разводе более возрастные, могут иметь детей, внуков, которые помогают им с выплатой кредита. 

Не женатые/ в гражданском браке чаще молодежь, уровень дохода которой ниже.

## Общий вывод
<a id="conclusion"></a>

<div class="alert alert-block alert-info">
В ходе анализа был составлен индекс оценки возвращаемости кредитов. В индексе 0 означает, что ни один клиент в группе не имел задолженностей с выплатой кредита, 1 - все клиенты из группы имели задолженности. Данные индекс был подсчитан для разных категорий клиентов.<br>
    <br>
Клиенты были разделены на группы на основании разных параметров: <br>
        * По наличию детей;<br> 
        * По уровню дохода;  <br>
        * По семейному положению;  <br>
        * По цели получения кредита;  <br>
    <br>
    По <b>наличию детей</b> клиенты были разделены на 3 категории:<br> 1) бездетные (0 детей); <br>2) малодетные (1-2); <br>3) многодетные;<br>
    <br>
    По <b>уровню дохода</b> на 4: <br> 1) бедные; <br> 2) ниже среднего; <br> 3) выше среднего; <br> 4) богатые;<br>
    <br>
    По <b>семейному положению</b> использовали предложенное деление: <br> 1) в разводе; <br>2) вдовец/вдова;<br> 3) гражданский брак; <br> 4) женат/замужем;<br> 5) не женат/не замужем;<br>
    <br>
    По <b>цели получения кредита</b> произвели лемматизацию и раздели клиентов на 4 категории: <br> 1) покупка жилья;<br> 2) сделка с автомобилем; <br> 3) получение образования; <br> 4) проведение свадьбы;<br>
    <br>
    В ходе исследования мы сделали выводы, что <b>семейное положение, наличие детей и цель получения кредита значительно влияют</b> на его возвращаемость. <b>Уровень дохода</b> также влиет на возвращаемость кредита, но в меньшей степени.<br>
    <br>
    <b>Наиболее надежной</b> группой являются люди, которые находятся или находились в браке, которые берут кредит с целью приобретения недвижимости. Среди них будут в приоритете те, кто не имеет детей, т.к. они несут меньшуют финансовую нагрузку.  Чуть менее надежными являются многодетные семьи с 3 и более детьми, т.к. это уже люди старшего возраста с более стабильными источниками дохода. Таким образом большее внимание нужно уделять следующим категориям людей: 1) молодые семьи без детей; 2) семьи с 3 и более детьми; <br> 
    <br>
    <b>Менее надежной</b> является группа неженатых, бездетных, берущих кредит на проведение свадьбы. В России принято дарить деньги на свадьбу, что создаёт финансовую подушку после её проведения. <br>
    <br>
    <b>Наиболее опасной</b> является группа неженатых/незамужних клиентов, клиентов живущих в гражданском браке, имеющих 1-2 детей, берущих кредит для получения образования либо на покупку автомобиля. Получение образования говорит о том, что клиент либо ещё не имеет работы, либо пытается её сменить, что означет нестабильность его финансового положения. Автомобиль является элементом статуса, и люди часто пытатся его поднять, не имея на это возможности. Первые дети появляются в раннем возрасте, когда люди только начинают свою карьеру и ещё не достигают пика зароботка. Люди не в браке не имеют той поддержки с небес, которая посылается праведным семьям, создавшим брак в лоне церкви. Шутка:)<br>
    <br>